In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#Importing all necessary libraries

import os
import shutil
import pandas as pd
import numpy as np
import keras_preprocessing
from keras.callbacks import ModelCheckpoint
from keras_preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Conv2D, LeakyReLU, MaxPooling2D, concatenate, AveragePooling2D, Flatten
from keras.layers import Dense, SpatialDropout1D, Dropout

Using TensorFlow backend.


In [0]:
#Directory assignments

base_dir='drive/My Drive/FlipkartGRID'
train_dir=os.path.join(base_dir, 'trainImages')

In [0]:
for image_name in os.listdir(os.path.join(base_dir, 'extratrainImages')):
  os.rename(src=os.path.join(os.path.join(base_dir, 'extratrainImages'), image_name), dst=os.path.join(train_dir, image_name))

In [0]:
#Reading training.csv

train_df=pd.read_csv(os.path.join(base_dir, 'training.csv'))

In [0]:
#Checking consistency(counts) of train_df and actual images in directory

print('Entries in train_df: '+str(len(train_df)))
print('Actual Number of images in directory: ' + str(len(os.listdir(train_dir))))

In [0]:
#Removing the extra images from directory, which donot have entries in train_df:

training_images_directory=os.listdir(train_dir)

for image_name in training_images_directory:
    if image_name in train_df['image_name'].tolist():
        continue
    else:
        print('Removing image with ID: '+ str(image_name))
        os.rename(src=os.path.join(train_dir, image_name), dst=os.path.join('drive/My Drive/FlipkartGRID/extratrainImages', image_name))

In [0]:
#Image flow from directory

datagen=ImageDataGenerator(rescale=1.0/255, validation_split=0.25)

train_generator=datagen.flow_from_dataframe(
                                            dataframe=train_df,
                                            directory=train_dir,
                                            x_col='image_name',
                                            y_col=['x1', 'x2', 'y1', 'y2'],
                                            subset='training',
                                            target_size=(128, 128),
                                            color_mode='grayscale',
                                            batch_size= 100,
                                            shuffle=True,
                                            seed=1,
                                            class_mode='other'
                                           )

validation_generator=datagen.flow_from_dataframe(
                                             dataframe=train_df,
                                             directory=train_dir,
                                             x_col='image_name',
                                             y_col=['x1', 'x2', 'y1', 'y2'],
                                             subset='validation',
                                             target_size=(128, 128),                                                                                        
                                             color_mode='grayscale',
                                             batch_size= 100,
                                             shuffle=True,
                                             seed=1,
                                             class_mode='other'
                                            )

Found 10500 images.
Found 3500 images.


In [0]:
#model2_2 architecture (light)

from keras.layers import SpatialDropout2D
inputTensor=Input((128, 128, 1))

outputTensor=Conv2D(kernel_size=(20, 20), filters=64, activation=None)(inputTensor)
outputTensor=LeakyReLU(alpha=0.2)(outputTensor)
outputTensor=MaxPooling2D(pool_size=(2, 2))(outputTensor)
outputTensor=SpatialDropout2D(0.1)(outputTensor)

outputTensor=Conv2D(kernel_size=(40, 40), filters=128, activation=None)(outputTensor)
outputTensor=MaxPooling2D(pool_size=(2, 2))(outputTensor)

outputTensor=Flatten()(outputTensor)

outputTensor=Dense(units=2048, activation=None)(outputTensor)
outputTensor=LeakyReLU(alpha=0.2)(outputTensor)

outputTensor=Dense(units=256, activation=None)(outputTensor)
outputTensor=LeakyReLU(alpha=0.2)(outputTensor)
outputTensor=Dropout(0.1)(outputTensor)

outputTensor=Dense(units=20, activation=None)(outputTensor)
outputTensor=LeakyReLU(alpha=0.2)(outputTensor)

outputTensor=Dense(units=4, activation=None)(outputTensor)

model2_2=Model(inputs=inputTensor, outputs=outputTensor)

model2_2.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 64)      25664     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 109, 109, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
spatial_dropout2d_1 (Spatial (None, 54, 54, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)    

In [0]:
#Compilation cell

modelcheckPoint=ModelCheckpoint(filepath='drive/My Drive/model2_2.h5',
                                monitor='val_loss',
                                mode='min',
                                save_best_only=True,
                                save_weights_only=False,
                               )

model2_2.compile(optimizer='adam',
              metrics=['accuracy'],
              loss=['mse'])

In [0]:
#Fitting cell

model2_2.fit_generator(train_generator,
                    steps_per_epoch=105,
                    epochs=80, 
                    callbacks=[modelcheckPoint],
                    validation_data=validation_generator,
                    validation_steps=35,
                    verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/80
105/105 [==============================] - 2937s 28s/step - loss: 36792.6963 - acc: 0.8622 - val_loss: 6727.8794 - val_acc: 0.8994
Epoch 2/80
105/105 [==============================] - 213s 2s/step - loss: 7817.6908 - acc: 0.9043 - val_loss: 5538.6072 - val_acc: 0.8994
Epoch 3/80
105/105 [==============================] - 208s 2s/step - loss: 6332.5313 - acc: 0.9039 - val_loss: 4678.5770 - val_acc: 0.8997
Epoch 4/80
105/105 [==============================] - 211s 2s/step - loss: 5603.6301 - acc: 0.9026 - val_loss: 6801.6491 - val_acc: 0.8957
Epoch 5/80
105/105 [==============================] - 207s 2s/step - loss: 4880.5984 - acc: 0.9016 - val_loss: 3008.6631 - val_acc: 0.8994
Epoch 6/80
105/105 [==============================] - 207s 2s/step - loss: 4743.0759 - acc: 0.9028 - val_loss: 3138.2360 - val_acc: 0.9006
Epoch 7/80
105/105 [==============================] - 208s 2s/step - loss: 3644.1408 - acc: 0.9049 - val_loss: 2878

In [0]:
model2.save(os.path.join(base_dir, 'model2.h5'))